# Imports

In [1]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic lines
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import re

# Load data

In [3]:
! ls ../data

all_games_df_2023-01_to_2024-05.csv  all_games_df.csv  all_games_raw.json


In [4]:
filename = "all_games_df_2023-01_to_2024-05.csv"
filepath = os.path.join("..","data", filename)
all_games_df_raw = pd.read_csv(filepath)

In [5]:
all_games_df_raw

,date,start_time,end_time,time_class,time_control,rated,rules,url,moves,final_position_fen,opening_code,opening_name,white_username,black_username,result,white_rating,black_rating,white_result,black_result
0,2023-01-07,2023-01-07 08:17:06,2023-01-07 14:35:53,daily,1/604800,True,chess,https://www.chess.com/game/daily/437377841,1. e4 {[%clk 158:31:38] 1... c6 {[%clk 163:21:...,2r1k2r/pp3ppp/q7/4pb2/1b1P4/1P2B3/1P3PPP/K2R2N...,B10,Caro-Kann-Defense-Hillbilly-Attack-2...d5,sylvainau,JammyNinja,0-1,1298,1461,checkmated,win
1,2023-01-14,2023-01-14 11:34:48,2023-01-14 15:09:28,daily,1/604800,True,chess,https://www.chess.com/game/daily/448245095,1. e4 {[%clk 167:35:57] 1... c5 {[%clk 165:23:...,r2qr3/1b2bpkp/p2p1n2/1p4B1/3Q4/2P2N2/PP3PPP/3R...,B22,Alapin-Sicilian-Defense-2...Nc6-3.d4,JammyNinja,Timkee,0-1,1429,1234,timeout,win
2,2023-01-30,2023-01-30 17:32:36,2023-01-30 07:50:02,daily,1/604800,True,chess,https://www.chess.com/game/daily/436095119,1. e4 {[%clk 167:38:55] 1... c6 {[%clk 163:21:...,rr4k1/3n2pp/2p2p2/p2pp3/P7/1P5P/1RPPNPP1/5RK1 ...,B10,Caro-Kann-Defense-2.Nf3-d5-3.exd5-cxd5,ELNAHAS4,JammyNinja,0-1,1427,1449,timeout,win
3,2023-01-30,2023-01-30 21:29:25,2023-01-30 21:39:52,blitz,300,True,chess,https://www.chess.com/game/live/68853253455,1. Nf3 {[%clk 0:04:54.9] 1... d5 {[%clk 0:04:5...,8/8/4K3/8/8/8/4Q3/6k1 b - -,A09,Reti-Opening-Reti-Gambit-Accepted,ivangard57,JammyNinja,1-0,1022,1030,win,timeout
4,2023-02-07,2023-02-07 20:18:19,2023-02-07 19:25:50,daily,1/604800,True,chess,https://www.chess.com/game/daily/472634081,1. c4 {[%clk 164:52:18] 1... e5 {[%clk 125:36:...,rnbqk1nr/pppp1ppp/8/2b1p3/2P5/2N2P2/PP1PP1PP/R...,A21,English-Opening-Reversed-Sicilian-Variation,ajbouchie,JammyNinja,1/2-1/2,1584,1449,agreed,agreed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3203,2024-05-31,2024-05-31 12:33:22,2024-05-31 12:35:40,bullet,60,True,chess,https://www.chess.com/game/live/110898007115,1. e4 {[%clk 0:01:00] 1... d5 {[%clk 0:00:59.9...,8/8/8/RQ6/1k6/4N3/P5PK/3N4 b - -,B01,Scandinavian-Defense-Mieses-Kotrc-Main-Line-4....,Ingbing,JammyNinja,1-0,1356,1350,win,checkmated
3204,2024-05-31,2024-05-31 12:35:50,2024-05-31 12:37:37,bullet,60,True,chess,https://www.chess.com/game/live/110898034915,1. e4 {[%clk 0:01:00] 1... e6 {[%clk 0:01:00] ...,r2q1rk1/p3bppp/1p6/3RP3/1P3NQn/P7/1B3PPP/2R3K1...,C00,French-Defense-Queens-Knight-Variation-2...d5,JammyNinja,nhathuy82vn,1-0,1358,1360,win,timeout
3205,2024-05-31,2024-05-31 12:37:42,2024-05-31 12:39:28,bullet,60,True,chess,https://www.chess.com/game/live/110898057561,1. e4 {[%clk 0:01:00] 1... e5 {[%clk 0:01:00] ...,6rk/1p6/p2p4/2p5/3pP2q/P2P4/1PP5/R4RK1 w - -,C50,Giuoco-Piano-Game-4.Nc3,JammyNinja,21To25,0-1,1350,1347,checkmated,win
3206,2024-05-31,2024-05-31 13:49:35,2024-05-31 13:51:50,bullet,60,True,chess,https://www.chess.com/game/live/110902287015,1. e4 {[%clk 0:01:00] 1... d5 {[%clk 0:01:00] ...,5Q2/5R1k/7p/3pp2P/3P2p1/1P1N2P1/8/7K b - -,B01,Scandinavian-Defense-2.e5,Competent_Checkmate,JammyNinja,1-0,1356,1342,win,checkmated


In [6]:
test_game = all_games_df_raw.query("time_class == 'bullet'").iloc[1]
test_moves = test_game["moves"]
test_game

date                                                         2023-09-13
start_time                                          2023-09-13 19:47:02
end_time                                            2023-09-13 19:48:51
time_class                                                       bullet
time_control                                                       60+1
rated                                                              True
rules                                                             chess
url                         https://www.chess.com/game/live/88373428625
moves                 1. e4 {[%clk 0:01:01] 1... e5 {[%clk 0:01:01] ...
final_position_fen    2kr3r/1pp1q1pp/p1n2n2/2b1p3/4p3/1BN5/PPPPQPPP/...
opening_code                                                        C47
opening_name                                          Four-Knights-Game
white_username                                               JammyNinja
black_username                                                in

# Add new columns

In [7]:
def process_moves(row):
    """
        Looks at 'moves' column which is pgn string, extracts each players moves and when they castled
        ['move_numbers', 'white_moves', 'white_clock', 'black_moves', 'black_clock', 
        'w_cstl_side', 'w_cstl_move', 'b_cstl_side', 'b_cstl_move']
    """
    #.apply() complained that it was float when didn't manually convert to string?
    moves = str(row["moves"])
    moves = moves.replace("{","")

    move_nums = moves.split()[::8]
    white_moves = moves.split()[1::8]
    black_moves = moves.split()[5::8]      
    
    white_clock = [m[:-1] for m in moves.split()[3::8] ]
    black_clock = [m[:-1] for m in moves.split()[7::8] ]
     
    if len(black_moves) < len(white_moves):
        black_moves.append(np.nan)
        black_clock.append(np.nan)
    
    #deal with castling
    w_castled, w_castled_side = 0,np.nan
    b_castled, b_castled_side = 0, np.nan
    
    for move_num, w_move, b_move in zip(move_nums, white_moves, black_moves):
        move_num = int(move_num[:-1]) #7. -> 7
        
        if w_move == "O-O":
            w_castled_side = "King"
            w_castled = move_num
        elif w_move == "O-O-O":
            w_castled_side = "Queen"
            w_castled = move_num
            
        if b_move == "O-O":
            b_castled_side = "King"
            b_castled = move_num
        elif b_move == "O-O-O":
            b_castled_side = "Queen"
            b_castled = move_num
            
#     print(f"white castled {w_castled_side} side on move {w_castled}")
#     print(f"black castled {b_castled_side} side on move {b_castled}")
    
    df_dict = {
        "move_numbers" : move_nums,
        "white_moves"  : white_moves,
        "white_clock" : white_clock,
        "black_moves"  : black_moves,
        "black_clock" : black_clock,
        
        "w_cstl_side" : w_castled_side,
        "w_cstl_move" : w_castled,        
        "b_cstl_side" : b_castled_side,
        "b_cstl_move" : b_castled
    }
    
    return df_dict
    
process_moves(all_games_df_raw.iloc[0])

{'move_numbers': ['1.',
  '2.',
  '3.',
  '4.',
  '5.',
  '6.',
  '7.',
  '8.',
  '9.',
  '10.',
  '11.',
  '12.',
  '13.',
  '14.',
  '15.',
  '16.',
  '17.',
  '18.',
  '19.',
  '20.'],
 'white_moves': ['e4',
  'Bc4',
  'exd5',
  'Bb3',
  'd4',
  'Qe2',
  'Qd3',
  'Bg5',
  'Qe3',
  'Nd2',
  'c3',
  'Rd1',
  'axb3',
  'Ke2',
  'Bxe3',
  'Kd2',
  'cxd4',
  'Kc1',
  'Kb1',
  'Ka1'],
 'white_clock': ['158:31:38',
  '166:44:09',
  '158:41:19',
  '159:21:21',
  '157:23:04',
  '159:01:58',
  '86:08:40',
  '64:25:54',
  '144:09:43',
  '158:35:55',
  '156:16:18',
  '164:50:45',
  '86:22:55',
  '0:00:00',
  '134:31:06',
  '161:17:27',
  '162:47:06',
  '164:59:10',
  '109:12:03',
  '166:46:52'],
 'black_moves': ['c6',
  'd5',
  'cxd5',
  'Nf6',
  'Nc6',
  'Nxd4',
  'e5',
  'Be6',
  'Qa5+',
  'Ne4',
  'Nxd2',
  'N2xb3',
  'Nc2+',
  'Nxe3',
  'Qb5+',
  'd4',
  'Bb4+',
  'Rc8+',
  'Bf5+',
  'Qa6#'],
 'black_clock': ['163:21:45',
  '135:49:04',
  '76:17:28',
  '157:17:31',
  '105:37:31',
  '56:27:2

In [8]:
def process_winner_rating(row):
    
    winner = "white" if row["result"] == "1-0" else "black" if row["result"] == "0-1" else "DRAW"
    
    w_rating, b_rating = row["white_rating"], row["black_rating"] #reduce line size
    
    higher_rated = "white" if w_rating > b_rating else "black" #assumes draw is good for black
    
    user_colour = "white" if row["white_username"] == "JammyNinja" else "black"
        
    def did_user_win(user = "JammyNinja"):
        if winner == "white" and row["white_username"] == user:
            return True
        if winner == "black" and row["black_username"] == user:
            return True
        if winner == "draw":
            return "DRAW"
        return False
    
    user_win = did_user_win()
    
    winner_rating_diff = w_rating - b_rating if winner == "white" else b_rating - w_rating
    user_rating_diff = w_rating - b_rating if user_colour == "white" else b_rating - w_rating
        
    return {
        "higher_rated_colour" : higher_rated,
        "winner"       : winner,
        "winner_rating_diff"  : winner_rating_diff,
        "user_rating_diff" : user_rating_diff,
        "user_win"     : user_win,
        "user_colour" : user_colour
    }
process_winner_rating(all_games_df_raw.iloc[4])

{'higher_rated_colour': 'white',
 'winner': 'DRAW',
 'winner_rating_diff': -135,
 'user_rating_diff': -135,
 'user_win': False,
 'user_colour': 'black'}

In [9]:
def user_castled(row):
#         print(row)
        user_castled = False
        opp_castled = False

        w_cstl_move = row["w_cstl_move"]
        b_cstl_move = row["b_cstl_move"]

        if row["user_colour"] == "white":
            if w_cstl_move > 0:
                user_castled = True
                user_cstl_diff = w_cstl_move - b_cstl_move  
            if b_cstl_move > 0:
                opp_castled = True
        else: #user is black
            if b_cstl_move > 0:
                user_castled = True
                user_cstl_diff = b_cstl_move - w_cstl_move 
            if w_cstl_move > 0:
                opp_castled = True
                
        
        #create columns to do with who castled first         
        #null if user didn't castle
        if not user_castled:
            user_cstl_diff = np.nan
        #WHAT IF I CASTLE AND THEY DONT - cant subtract?
        
        return {
            "user_castled" : user_castled,
            "opp_castled" : opp_castled,
#             "user_cstl_diff" : user_cstl_diff,
#             "winner_cstl_diff" : win_cstl_diff
        }
# user_castled(all_games_df.iloc[80])
# display(all_games_df.iloc[80])

In [10]:
test_game

date                                                         2023-09-13
start_time                                          2023-09-13 19:47:02
end_time                                            2023-09-13 19:48:51
time_class                                                       bullet
time_control                                                       60+1
rated                                                              True
rules                                                             chess
url                         https://www.chess.com/game/live/88373428625
moves                 1. e4 {[%clk 0:01:01] 1... e5 {[%clk 0:01:01] ...
final_position_fen    2kr3r/1pp1q1pp/p1n2n2/2b1p3/4p3/1BN5/PPPPQPPP/...
opening_code                                                        C47
opening_name                                          Four-Knights-Game
white_username                                               JammyNinja
black_username                                                in

In [13]:
def pieces_final_position(game):
    """
    starts with eighth rank, first file to eighth
    white is uppercase, black lowercase
    p: pawn, r: rook, n: knight, b: bishop, q : queen, k : king 
    """
    fen = game["final_position_fen"]
    fin_pos = fen.split()[0]
    
    board = fin_pos.split("/")
    ttl_white_pcs, ttl_black_pcs = 0,0
    
    #white pieces are uppercase, black lowercase
    w_pcs = [piece for piece in "".join(board) if piece.isupper()]
    b_pcs = [piece for piece in "".join(board) if piece.islower()]
    #count total pieces (including king), and also count just the pawns
    ttl_white_pcs = len(w_pcs)
    ttl_black_pcs = len(b_pcs)
    ttl_white_pwns = len([p for p in w_pcs if p == "P"])
    ttl_black_pwns = len([p for p in b_pcs if p == "p"])

    #create material counts
    def sum_piece_values(pcs):
        """
        Accepts as input b_pcs or w_bcs and outputs the total value of what's given.
        """
        # dictionary for piece values
        chess_piece_values = {
            'P': 1,  # Pawn
            'N': 3,  # Knight
            'B': 3,  # Bishop
            'R': 5,  # Rook
            'Q': 9,  # Queen
            'K': 0,   # King (often considered to have infinite value, but in practical terms, it is 0 for valuation purposes)
        }
        count = 0
        for i in pcs:
            count += chess_piece_values[i.upper()]
        return count
    
    w_material_count=sum_piece_values(w_pcs)
    b_material_count=sum_piece_values(b_pcs)
    
    if game.winner == "white":
        win_pcs, lose_pcs = w_pcs, b_pcs
        winner_ttl_pieces_count = ttl_white_pcs
        loser_ttl_pieces_count = ttl_black_pcs
        winner_pawns_count = ttl_white_pwns
        loser_pawns_count = ttl_black_pwns
        winner_material_ttl = w_material_count
        loser_material_ttl = b_material_count        
        
    else: #includes DRAWs
        win_pcs, lose_pcs = b_pcs, w_pcs
        winner_ttl_pieces_count = ttl_black_pcs
        loser_ttl_pieces_count = ttl_white_pcs
        winner_pawns_count = ttl_black_pwns
        loser_pawns_count = ttl_white_pwns
        winner_material_ttl = b_material_count
        loser_material_ttl = w_material_count
    
    #count of just the major/minor pieces, -1 to not include King,    
    winner_pieces_only_count = winner_ttl_pieces_count - winner_pawns_count -1
    loser_pieces_only_count = loser_ttl_pieces_count - loser_pawns_count -1
    
    output = {
        #pieces on the board including king
        "winner_ttl_pieces_count" : winner_ttl_pieces_count, 
        "loser_ttl_pieces_count" : loser_ttl_pieces_count,
#       #count pawns
        "winner_pawns_count" : winner_pawns_count, 
        "loser_pawns_count" : loser_pawns_count,
        #count of just the major/minor pieces
        "winner_pieces_only_count" : winner_pieces_only_count,
        "loser_pieces_only_count" : loser_pieces_only_count,
        "pieces_count_diff" : winner_pieces_only_count - loser_pieces_only_count,
        "pawns_count_diff" : winner_pawns_count - loser_pawns_count,
        #count the material worth
        "winner_material_ttl" : winner_material_ttl,
        "loser_material_ttl" : loser_material_ttl,
        #might as well throw the pieces in too
        "winner_pcs" : win_pcs,
        "loser_pcs"  : lose_pcs,
    }
    
    return output

test_game = all_games_df_raw.query("time_class == 'bullet'").iloc[9]
# print(test_game.final_position_fen)
# pieces_final_position(test_game)

In [72]:
def add_new_columns(games_df, printing=True):
    
    orig_cols = games_df.columns
    
    #uses winner_rating function to calculate the winner and rating swing
    win_rating_cols = games_df.apply(process_winner_rating, axis="columns", result_type="expand")
    
    #uses process moves function to break down the moves into w/b, and when castled
    move_cols =       games_df.apply(process_moves, axis="columns",result_type="expand")
    
    #add the above columns for now, so that the below funcs know who the winner was
    games_df = pd.concat([win_rating_cols, move_cols, games_df], axis=1)

    #create final position columns, gets the material left on the board
    final_pos_cols = games_df.apply(pieces_final_position, axis="columns",result_type="expand")
    
    #now calculate castle cols using some of the above columns
    cstl_cols =       games_df.apply(user_castled, axis="columns", result_type="expand")
    games_df = pd.concat([games_df, cstl_cols, final_pos_cols], axis=1)
    
    if printing:
        print("Created win/rating columns:\n" + "\n".join([f"- {col}" for col in win_rating_cols]))
        print("Created moves columns:\n" + "\n".join([f"- {col}" for col in move_cols]))
        print("Created final position columns:\n" + "\n".join([f"- {col}" for col in final_pos_cols]))
        print("Created castling columns:\n" + "\n".join([f"- {col}" for col in cstl_cols]))
        print("\nStarted with:\n" + "\n".join([f"- {col}" for col in orig_cols]))
    return games_df
    
all_games_df = add_new_columns(all_games_df_raw)
all_games_df.sample(2)

Created win/rating columns:
- higher_rated_colour
- winner
- winner_rating_diff
- user_rating_diff
- user_win
- user_colour
Created moves columns:
- move_numbers
- white_moves
- white_clock
- black_moves
- black_clock
- w_cstl_side
- w_cstl_move
- b_cstl_side
- b_cstl_move
Created final position columns:
- winner_ttl_pieces_count
- loser_ttl_pieces_count
- winner_pawns_count
- loser_pawns_count
- winner_pieces_only_count
- loser_pieces_only_count
- pieces_count_diff
- pawns_count_diff
- winner_material_ttl
- loser_material_ttl
- winner_pcs
- loser_pcs
Created castling columns:
- user_castled
- opp_castled

Started with:
- date
- start_time
- end_time
- time_class
- time_control
- rated
- rules
- url
- moves
- final_position_fen
- opening_code
- opening_name
- white_username
- black_username
- result
- white_rating
- black_rating
- white_result
- black_result


,higher_rated_colour,winner,winner_rating_diff,user_rating_diff,user_win,user_colour,move_numbers,white_moves,white_clock,black_moves,...,winner_pawns_count,loser_pawns_count,winner_pieces_only_count,loser_pieces_only_count,pieces_count_diff,pawns_count_diff,winner_material_ttl,loser_material_ttl,winner_pcs,loser_pcs
1669,black,black,40,40,True,black,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[e4, Nf3, Bc4, O-O, d3, a4, h3, Nbd2, c3, Qb3,...","[0:03:00, 0:02:59.3, 0:02:58.6, 0:02:57.6, 0:0...","[e5, Nc6, Bc5, d6, a6, Bg4, Bh5, Nf6, O-O, Bb6...",...,3,3,3,0,3,0,24,3,"[p, k, n, p, p, q, q]","[P, P, P, K]"
409,black,black,72,72,True,black,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[e4, Bc4, d3, dxc4, b3, Bb2, Nc3, Nf3, h3, gxf...","[0:01:00, 0:00:59.4, 0:00:59.3, 0:00:58.6, 0:0...","[e5, b5, bxc4, c5, d6, Nc6, Nd4, Bg4, Nxf3+, B...",...,7,5,5,5,0,2,32,30,"[r, r, k, p, p, p, p, p, p, q, b, b, p]","[N, P, P, Q, P, P, B, P, K, R, R]"


In [37]:
all_games_df.columns

Index(['higher_rated_colour', 'winner', 'winner_rating_diff',
       'user_rating_diff', 'user_win', 'user_colour', 'move_numbers',
       'white_moves', 'white_clock', 'black_moves', 'black_clock',
       'w_cstl_side', 'w_cstl_move', 'b_cstl_side', 'b_cstl_move', 'date',
       'start_time', 'end_time', 'time_class', 'time_control', 'rated',
       'rules', 'url', 'moves', 'final_position_fen', 'opening_code',
       'opening_name', 'white_username', 'black_username', 'result',
       'white_rating', 'black_rating', 'white_result', 'black_result',
       'user_castled', 'opp_castled', 'winner_ttl_pieces_count',
       'loser_ttl_pieces_count', 'winner_pawns_count', 'loser_pawns_count',
       'winner_pieces_only_count', 'loser_pieces_only_count',
       'pieces_count_diff', 'pawns_count_diff', 'winner_material_ttl',
       'loser_material_ttl', 'winner_pcs', 'loser_pcs'],
      dtype='object')

In [38]:
test_game = all_games_df.query("time_class == 'bullet'").iloc[10]
test_game

higher_rated_colour                                                     white
winner                                                                   DRAW
winner_rating_diff                                                        -22
user_rating_diff                                                          -22
user_win                                                                False
user_colour                                                             black
move_numbers                [1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...
white_moves                 [e4, Nc3, Nf3, Bc4, Bxe6, O-O, d4, dxe5, Qxd8,...
white_clock                 [0:01:00, 0:00:59.9, 0:00:59.7, 0:00:59.6, 0:0...
black_moves                 [e5, d6, Nf6, Be6, fxe6, Be7, O-O, dxe5, Rxd8,...
black_clock                 [0:01:00, 0:00:59.4, 0:00:58, 0:00:56, 0:00:55...
w_cstl_side                                                              King
w_cstl_move                                                     

## order the columns

In [64]:
all_cols_order = [
    #meta
    'date', 'start_time', 'end_time',
    'time_class', 'time_control', 'rated', 'rules',
    'url', 'moves',
    'opening_code', 'opening_name',
    'white_username', 'black_username',
    'white_clock', 'black_clock',
    
    #ratings/results
    'white_rating', 'black_rating',
    'result', 'white_result', 'black_result',
    'higher_rated_colour', 'winner', 'winner_rating_diff',
    'user_rating_diff', 'user_win', 'user_colour', 'move_numbers',
     
    #moves
    'white_moves',  'black_moves',
    'w_cstl_side', 'w_cstl_move', 
    'b_cstl_side', 'b_cstl_move',
    'user_castled', 'opp_castled',
    
    #final position
     'final_position_fen',
    'winner_ttl_pieces_count','loser_ttl_pieces_count',
    'winner_pawns_count', 'loser_pawns_count',
    'winner_pieces_only_count', 'loser_pieces_only_count',
    'pieces_count_diff', 'pawns_count_diff',
    'winner_material_ttl', 'loser_material_ttl',
    'winner_pcs', 'loser_pcs'
]

cols_pre = all_games_df.columns
all_games_df = all_games_df[all_cols_order]

removed_cols = [col for col in all_games_df.columns if col not in cols_pre]
if removed_cols:
    print("You have excluded the following columns:\n" + "\n".join([f"- {col}" for col in removed_cols]))

## Save to file

In [70]:
filename = "all_games_all_cols.csv"
all_games_df.to_csv(os.path.join("..","data",filename), index=False)

# Selecting data

### Exclude rows

In [42]:
selected_games_df = all_games_df.copy()

#exclude any chess960 games
selected_games_df = all_games_df.query("rules == 'chess'")

#exclude unrated games
selected_games_df = selected_games_df.query("rated == True")

#exclude daily games
selected_games_df = selected_games_df.query("time_class != 'daily'")

#exclude excess rating diff
diff_max = 400
selected_games_df = selected_games_df.query(f"abs(winner_rating_diff) <= {diff_max}")

print(f"Excluded {len(all_games_df) - len(selected_games_df)} rows due to constraints")
print(f"Leaves {len(selected_games_df)} games to work with")

draws_df = selected_games_df.query("winner=='DRAW'") #result=='1/2-1/2'
print(f"Of which {len(draws_df)} are draws")

selected_games_df.sample(3)

Excluded 104 rows due to constraints
Leaves 3104 games to work with
Of which 124 are draws


,higher_rated_colour,winner,winner_rating_diff,user_rating_diff,user_win,user_colour,move_numbers,white_moves,white_clock,black_moves,...,winner_pawns_count,loser_pawns_count,winner_pieces_only_count,loser_pieces_only_count,pieces_count_diff,pawns_count_diff,winner_material_ttl,loser_material_ttl,winner_pcs,loser_pcs
1073,white,black,-10,-10,True,black,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[e4, Bc4, Nf3, Nc3, a3, d3, Nxd4, O-O, Be3, Bx...","[0:03:00, 0:02:57.4, 0:02:55.7, 0:02:51.7, 0:0...","[e5, Bc5, Nc6, d6, a6, Nd4, Bxd4, Ne7, Bb6, cx...",...,7,7,4,3,1,0,29,26,"[r, r, k, p, p, p, p, p, q, p, p, b]","[Q, P, P, P, P, P, P, P, R, R, K]"
380,white,white,24,24,True,white,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.]","[d4, f4, e3, exd4, c3, Bd3, Nf3, O-O, Bd2, Be1...","[0:03:00, 0:02:58.3, 0:02:57, 0:02:56.9, 0:02:...","[e6, c5, cxd4, Nc6, Nf6, d6, Be7, a6, O-O, Nb8...",...,7,7,7,7,0,0,38,38,"[P, P, P, B, N, P, P, N, P, P, R, Q, B, R, K]","[r, n, b, q, r, k, p, b, p, p, p, p, p, p, n]"
2169,black,white,-8,8,False,black,"[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.,...","[d4, c4, Nf3, Nc3, e4, Be3, gxf3, Bxc4, Rc1, a...","[0:01:00, 0:00:59.3, 0:00:58.8, 0:00:58.3, 0:0...","[d5, Nc6, dxc4, Nf6, Bg4, Bxf3, e6, Bd6, a6, b...",...,2,5,2,4,-2,-3,14,25,"[Q, P, B, P, K]","[r, k, p, p, p, p, p, n, n, q]"


### Select columns

In [52]:
select_cols = [
        #metadata
        'date', 'url',
        'start_time', 'end_time', 
        'time_class', 'time_control',
#         'rated', 'rules', #used to filter out unwanted games
        'white_username', 'black_username', "user_colour",
        #ratings/result
        'result', 'winner', 'higher_rated_colour',
        'white_rating', 'black_rating', 
        'white_result', 'black_result',
        'winner_rating_diff', "user_rating_diff",
        #specific moves of the game
        'opening_name', 'opening_code',
        'w_cstl_side', 'w_cstl_move', 
        'b_cstl_side', 'b_cstl_move',
        'user_castled', 'opp_castled',
        #final position stats
        'winner_ttl_pieces_count', 'loser_ttl_pieces_count',
        'winner_pawns_count', 'loser_pawns_count',
        'winner_pieces_only_count', 'loser_pieces_only_count',
        'pieces_count_diff', 'pawns_count_diff', 
        'winner_material_ttl', 'loser_material_ttl',
        'winner_pcs', 'loser_pcs',
        #target
        'user_win' 
]

selected_games_df = selected_games_df[select_cols]

removed_cols = [col for col in all_games_df.columns if col not in selected_games_df.columns]
print("You have excluded the following columns:\n" + "\n".join([f"- {col}" for col in removed_cols]))



You have excluded the following columns:
- move_numbers
- white_moves
- white_clock
- black_moves
- black_clock
- rated
- rules
- moves
- final_position_fen


In [53]:
selected_games_df.sample(5)

,date,url,start_time,end_time,time_class,time_control,white_username,black_username,user_colour,result,...,loser_pawns_count,winner_pieces_only_count,loser_pieces_only_count,pieces_count_diff,pawns_count_diff,winner_material_ttl,loser_material_ttl,winner_pcs,loser_pcs,user_win
1649,2024-01-26,https://www.chess.com/game/live/100010665091,2024-01-26 12:23:51,2024-01-26 12:28:44,blitz,180,Calmetempete,JammyNinja,black,0-1,...,2,1,1,0,0,7,7,"[k, p, p, r]","[R, P, P, K]",True
204,2023-11-05,https://www.chess.com/game/live/92967047451,2023-11-05 23:44:21,2023-11-05 23:46:21,bullet,60,matheusibanez,JammyNinja,black,0-1,...,7,1,4,-3,-2,14,27,"[k, p, p, p, p, p, q]","[N, B, P, P, P, P, P, P, Q, P, K, R]",True
757,2023-12-17,https://www.chess.com/game/live/96574785967,2023-12-17 17:58:41,2023-12-17 18:00:44,bullet,60,JammyNinja,Adrian-82,white,0-1,...,7,4,4,0,0,27,23,"[n, q, b, r, k, p, p, p, p, p, p, p]","[B, P, P, P, P, B, P, P, R, P, R, K]",False
848,2023-12-29,https://www.chess.com/game/live/97607538321,2023-12-29 16:50:30,2023-12-29 16:55:31,blitz,180,gustavoo74,JammyNinja,black,1-0,...,1,4,2,2,3,26,9,"[R, Q, N, P, P, P, P, K, R]","[k, p, r, b]",False
223,2023-11-06,https://www.chess.com/game/live/92970704733,2023-11-06 00:49:37,2023-11-06 00:51:51,bullet,60,JammyNinja,Shervin_bond,white,1-0,...,5,3,5,-2,1,21,30,"[P, P, N, K, Q, P, B, P, P, P]","[r, k, r, b, p, p, p, p, p, b, q]",True
